# Conservation of invariants and code stability

This notebook aims at investigation the stability of the code as a function of input parameters and the conservation of invariants such as energy and momentum.


In [178]:
# some preliminary settings and functions

import sys
sys.path.append("/home/aunai/build_phare")
sys.path.append("/home/aunai/PHARE/pyphare/")
import os
import numpy as np
import pyphare
from pyphare.pharesee.hierarchy import finest_data
from pyphare.pharesee.hierarchy import hierarchy_from
from pyphare.pharesee.plotting import zoom_effect
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from dispersion import get_times
from glob import glob

%matplotlib widget


In [179]:
def plot_job(job, time, qty,**kwargs):
    path = job
    r = Run(path)
    t=time        
    B = r.GetB(t)
    #N = r.GetNi(t)
    #V = r.GetVi(t)
    if qty in ("Bx","By","Bz"):
        self = B
    if qty == "rho":
        self = N
    if qty in ("Vx", "Vy", "Vz"):
        self = V
    fig, (ax1,ax2) = plt.subplots(nrows=2, figsize=(8,6), sharex=True)
    colors = ["k", "b", "r"]
    for ilvl, lvl in self.patch_levels.items():
        for pidx, patch in enumerate(lvl.patches):
            pdata  = patch.patch_datas[qty]
            x,v = finest_data(pdata, ilvl, self)
            ax1.plot(x, v,label="lev{} - patch{}".format(ilvl, pidx),
                     marker='o', markersize=2, color=colors[ilvl])

    for ilvl,level in B.patch_levels.items():
        for patch in level.patches:
            dx = patch.dx
            x0 = patch.origin[0]
            x1 = (patch.box.upper[0]+1)*dx
            ax2.axvspan(x0, x1, color='b',ec='k', alpha=0.2,
                        ymin=ilvl/4, ymax=(ilvl+1)/4)

    ax1.set_title("{} at t = {}".format(job,t))

    ax1.set_ylim(kwargs.get("ylim",(-3.5, 3.5)))

    ax1.set_xlim(kwargs.get("xlim", (0,500)))

    return fig, ax1, ax2

In [482]:
def kinetic_energy(particles, kind="iso"):
    if kind=="iso":
        return 0.5*np.sum((particles.v[:,0]**2 + particles.v[:,1]**2 + particles.v[:,2]**2)* particles.weights)
    if kind=="perp":
        return 0.5*np.sum((particles.v[:,1]**2 + particles.v[:,2]**2)* particles.weights)
    if kind=="para":
        return 0.5*np.sum((particles.v[:,0]**2)* particles.weights)

def momentum(particles):
    m = 1
    return m*np.asarray([np.sum(v*particles.weights) for v in [particles.v[:,0], particles.v[:,1], particles.v[:,2]]])

In [486]:
def total_particles(parts, fun, lvlNbr=0, **kwargs):
    for ilvl, lvl in parts.patch_levels.items():
        if lvlNbr == ilvl:
            tot = 0.
            for ip, patch in enumerate(lvl.patches):
                keys = list(patch.patch_datas.keys())
                pdata = patch.patch_datas[keys[0]]
                particles= pdata.dataset
                per_patch = fun(particles, **kwargs)
                #L = (pdata.layout.box.upper[0]-pdata.layout.box.lower[0]+1)*pdata.layout.dl[0]
                tot += per_patch
    return tot    

In [483]:
def total_kinetic(parts, lvlNbr=0, kind="iso"):
    return total_particles(parts, kinetic_energy, lvlNbr, kind=kind)

def total_momentum(parts, lvlNbr=0):
    return total_particles(parts, momentum, lvlNbr)

def mag_energy(B, lvlNbr=0):
    for ilvl, lvl in B.patch_levels.items():
        if lvlNbr == ilvl:
            tot = 0.
            for ip, patch in enumerate(lvl.patches):
                pdata = patch.patch_datas["Bx"]
                bx = patch.patch_datas["Bx"].dataset[5:-5]
                bytmp = patch.patch_datas["By"].dataset[4:-4]
                bztmp = patch.patch_datas["Bz"].dataset[4:-4]
                by = 0.5*(bytmp[1:]+bytmp[:-1])
                bz = 0.5*(bztmp[1:]+bztmp[:-1])
                per_patch = np.sum((bx**2 + by**2 + bz**2)*0.5*pdata.layout.dl[0])
                L = (pdata.layout.box.upper[0]-pdata.layout.box.lower[0]+1)*pdata.layout.dl[0]
                tot +=  per_patch
        return tot

$$ E_B = \int_0^LB^2/2dx  = \sum_{i=0}^{n_x-1}B_i^2/2dx$$

$$E_k = \sum_{p=0}^{npart} 0.5*w_p*v_p^2 $$

In [484]:
def energies(path, kkind="iso"):
    r = Run(path)
    times = get_times(r.path+"/EM_B.h5")
    Bnrj = np.zeros_like(times)
    K = np.zeros_like(times)
    for it,t in enumerate(times):
        B = r.GetB(t)
        protons = r.GetParticles(t, "protons")
        Bnrj[it] = mag_energy(B)
        K[it] = total_kinetic(protons,kind=kkind)    
    return r, Bnrj, K, times

def avg_interval(t1, t2, times):
    return [np.argmin(np.abs(times-t)) for t in (t1,t2)]


def plot_energies_abs(times, Bnrj, K, t1, t2):
    fig, ax1 =plt.subplots()
    ax1.plot(times, Bnrj, label="magnetic energy")
    ax2 = plt.twinx(ax1)
    ax2.plot(times, K, label="total kinetic", color="r")
    fig.legend()
    ax1.axvspan(t1, t2, color="b", alpha=0.3)
    ax1.set_xlim((t1,times[-1]))
    plt.close(fig)

    
def plot_energies_rel(times, Bnrj, K, t1, t2):
    fig, ax = plt.subplots()
    it1, it2 = avg_interval(t1, t2, times)
    B0 = np.mean(Bnrj[it1:it2+1])
    K0 = np.mean(K[it1:it2+1])
    ax.plot(times, np.abs(Bnrj-B0)/B0*100, label="magnetic energy")
    ax.plot(times, np.abs(K-K0)/K0*100, label="total kinetic")
    ax.legend()
    ax.set_yscale("log")
    ax.set_xlabel("x")
    ax.set_title("relative (%) variation of the energies")
    ax.set_xlim((t1,times[-1]))
    plt.close(fig)

## run001 : test run to load and plot particle data

The run has a uniform density n=1, B=Bx=1, Vth=0.3, there are 400 cells with 100 particles per cell. See [input script](/home/aunai/phare_jobs/tests/conservation/run001/conserv.py)

In [287]:
r1, Bnrj1, K1, times1 = energies("/home/aunai/phare_jobs/tests/conservation/run001/")

In [301]:
plot_energies_abs(times1, Bnrj1, K1, 0,0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [302]:
plot_energies_rel(times1, Bnrj1, K1, 0,0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [191]:
mom = np.zeros_like(times)
for it,t in enumerate(times):
    protons = r.GetParticles(t, "protons")
    P = total_momentum(protons)
    mom[it] = np.sqrt(P[0]**2 + P[1]**2 + P[2]**2)

In [203]:
fig, ax = plt.subplots()
ax.plot(times, np.abs(mom - mom[0])/mom[0]*100, label="total mometum")
ax.legend()
ax.set_yscale("log")
ax.set_xlabel("t")
ax.set_title("relative (%) variation of the total momentum")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'relative (%) variation of the total momentum')

In [204]:
fig, ax = plt.subplots()
ax.plot(times, mom, label="total mometum")
ax.legend()
ax.set_xlabel("t")
ax.set_title("temporal variation of the total momentum amplitude")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'temporal variation of the total momentum amplitude')

##  Run002

The run has a uniform density n=1, B=Bx=1, Vth=0.3, there are 400 cells with 100 particles per cell. See [input script](/home/aunai/phare_jobs/tests/conservation/run002/conserv.py) It is the same run as run001 just evolved for a longer time

In [ ]:
r2, Bnrj2, K2, times2 = energies("/home/aunai/phare_jobs/tests/conservation/run002/")

In [304]:
plot_energies_abs(times2, Bnrj2, K2, 2, 5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [303]:
plot_energies_rel(times2, Bnrj2, K2, 2, 5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [218]:
plot_job(r.path, 40.000000, "By", xlim=(0,80), marker="none", ylim=(-0.1,0.1))

<ipython-input-179-b063a0291c60>:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1,ax2) = plt.subplots(nrows=2, figsize=(8,6), sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 800x600 with 2 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f096d6c7a00>)

## Run003

In [ ]:
r3, Bnrj3, K3, times3 = energies("/home/aunai/phare_jobs/tests/conservation/run003")

In [279]:
plot_energies_abs(times3, Bnrj3, K3, t1, t2)

<ipython-input-278-985c359aa819>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [280]:
plot_energies_rel(times3, Bnrj3, K3, t1, t2)

<ipython-input-278-985c359aa819>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run004

In [377]:
r4, Bnrj4, K4, times4 = energies("/home/aunai/phare_jobs/tests/conservation/run004")

In [378]:
plot_energies_abs(times4, Bnrj4, K4, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [379]:
plot_energies_rel(times4, Bnrj4, K4, t1, t2)

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run005

In [380]:
r5, Bnrj5, K5, times5 = energies("/home/aunai/phare_jobs/tests/conservation/run005")

In [381]:
plot_energies_abs(times5, Bnrj5, K5, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [382]:
plot_energies_rel(times5, Bnrj5, K5, t1, t2)

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run006

In [366]:
r6, Bnrj6, K6, times6 = energies("/home/aunai/phare_jobs/tests/conservation/run006")

In [367]:
plot_energies_abs(times6, Bnrj6, K6, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [368]:
plot_energies_rel(times6, Bnrj6, K6, t1, t2)

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run007

In [372]:
r7, Bnrj7, K7, times7 = energies("/home/aunai/phare_jobs/tests/conservation/run007")

In [373]:
plot_energies_abs(times7, Bnrj7, K7, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [327]:
plot_energies_rel(times7, Bnrj7, K7, t1, t2)

<ipython-input-300-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run008

In [374]:
r8, Bnrj8, K8, times8 = energies("/home/aunai/phare_jobs/tests/conservation/run008")
plot_energies_abs(times8, Bnrj8, K8, t1, t2)
plot_energies_rel(times8, Bnrj8, K8, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run009

In [369]:
r9, Bnrj9, K9, times9 = energies("/home/aunai/phare_jobs/tests/conservation/run009")
plot_energies_abs(times9, Bnrj9, K9, t1, t2)
plot_energies_rel(times9, Bnrj9, K9, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run010

In [351]:
r10, Bnrj10, K10, times10 = energies("/home/aunai/phare_jobs/tests/conservation/run010")
plot_energies_abs(times10, Bnrj10, K10, t1, t2)
plot_energies_rel(times10, Bnrj10, K10, t1, t2)

<ipython-input-300-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-300-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [354]:
plot_job(r.path, 49.000000, "By", xlim=(0,20), marker="none", ylim=(-0.1,0.1))

<ipython-input-179-b063a0291c60>:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1,ax2) = plt.subplots(nrows=2, figsize=(8,6), sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 800x600 with 2 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f09692d5520>)

## Run011

In [370]:
r11, Bnrj11, K11, times11 = energies("/home/aunai/phare_jobs/tests/conservation/run011")
plot_energies_abs(times11, Bnrj11, K11, t1, t2)
plot_energies_rel(times11, Bnrj11, K11, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run012

In [384]:
r12, Bnrj12, K12, times12 = energies("/home/aunai/phare_jobs/tests/conservation/run012")
plot_energies_abs(times12, Bnrj12, K12, t1, t2)
plot_energies_rel(times12, Bnrj12, K12, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run013

In [385]:
r13, Bnrj13, K13, times13 = energies("/home/aunai/phare_jobs/tests/conservation/run013")
plot_energies_abs(times13, Bnrj13, K13, t1, t2)
plot_energies_rel(times13, Bnrj13, K13, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run014

In [386]:
r14, Bnrj14, K14, times14 = energies("/home/aunai/phare_jobs/tests/conservation/run014")
plot_energies_abs(times14, Bnrj14, K14, t1, t2)
plot_energies_rel(times14, Bnrj14, K14, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run015

In [392]:
r15, Bnrj15, K15, times15 = energies("/home/aunai/phare_jobs/tests/conservation/run015")
plot_energies_abs(times15, Bnrj15, K15, t1, t2)
plot_energies_rel(times15, Bnrj15, K15, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run016

In [394]:
r16, Bnrj16, K16, times16 = energies("/home/aunai/phare_jobs/tests/conservation/run016")
plot_energies_abs(times16, Bnrj16, K16, t1, t2)
plot_energies_rel(times16, Bnrj16, K16, t1, t2)

<ipython-input-362-925fd584528c>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 =plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-362-925fd584528c>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Effect of the time step

In [406]:
fig, ax = plt.subplots()
for times, K, dt in ((times4, K4, "0.001"),
                     (times5, K5, "0.0005"),
                     (times6, K6, "0.002")):
    ax.plot(times, np.abs(K-K[0])/K[0]*100, label=dt)
ax.legend()
ax.set_title("kinetic energy evolution as a function of time for varying dt")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta K$ (%)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta K$ (%)')

In [407]:
fig, ax = plt.subplots()
for times, K, dt in ((times4, Bnrj4, "0.001"),
                     (times5, Bnrj5, "0.0005"),
                     (times6, Bnrj6, "0.002")):
    ax.plot(times, K-K[0], label=dt)
ax.legend()
ax.set_title("magnetic energy evolution as a function of time for varying dt")
ax.set_xlabel("t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 't')

## Effect of the interpolation order

In [408]:
fig, ax = plt.subplots()
for times, K, interp in ((times7, K7, "2"),
                     (times6, K6, "1"),
                     (times8, K8, "3")   ):
    ax.plot(times, np.abs(K-K[0])/K[0]*100, label=interp)
ax.legend()
ax.set_title("kinetic energy evolution as a function of time for varying interp order")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta K$ (%)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta K$ (%)')

In [409]:
fig, ax = plt.subplots()
for times, K, interp in ((times7, Bnrj7, "2"),
                     (times6, Bnrj6, "1"),
                     (times8, Bnrj8, "3")   ):
    ax.plot(times, np.abs(K-K[0])/K[0]*100, label=interp)
ax.legend()
ax.set_title("Magnetic energy evolution as a function of time for varying interp order")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta E_B$ (%)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta E_B$ (%)')

## Effect of the mesh size

In [410]:
fig, ax = plt.subplots()
for times, K, interp in ((times9, K9, "dx=0.1 (dt=0.0005)"),
                        (times6, K6, "dx= 0.2 (dt=0.002)"),
                        (times5, K5, "dx= 0.2 (dt=0.0005)"),
                        (times11, K11, "dx=0.3 (dt=0.002)")):
    ax.plot(times, np.abs(K-K[0])/K[0]*100, label=interp)
ax.legend()
ax.set_title("kinetic energy evolution as a function of time for varying dx")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta K$ (%)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta K$ (%)')

In [411]:
fig, ax = plt.subplots()
for times, K, interp in ((times9, Bnrj9, "dx=0.1 (dt=0.0005)"),
                        (times6, Bnrj6, "dx= 0.2 (dt=0.002)"),
                        (times5, Bnrj5, "dx= 0.2 (dt=0.0005)")):
    ax.plot(times, np.abs(K-K[0])/K[0]*100, label=interp)
ax.legend()
ax.set_title("Magnetic energy evolution as a function of time for varying dx")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta E_B$ (%)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta E_B$ (%)')

## Effect of the thermal velocity

In [539]:
paths = glob("/home/aunai/phare_jobs/tests/conservation/run017/*vth*")
runs_vth = {}
Bnrj_vth = {}
K_vth = {}
times_vth={}
vthdx = np.asarray(sorted([[float(x) for x in path.split("/")[-1].strip("vth").split("dx")] for path in paths],
               key=lambda x:x[1]))
paths = sorted(paths, key=lambda path: float(paths[0].split("/")[-1].strip("vth").split("dx")[1]))
for path in paths:
    runs_vth[path], Bnrj_vth[path], K_vth[path], times_vth[path] = energies(path)

In [576]:
fig, ax = plt.subplots()
for path, K in K_vth.items():
    if "dx0.1" in path:
        continue
    it1, it2 = avg_interval(3,4, times_vth[path])
    K0= np.mean(K[it1:it2+1])    
    vth = path.split("/")[-1].strip("vth").split("dx")[0]
    ax.plot(times_vth[path], np.abs(K-K0)/K0*100, label="vth={}".format(vth))
ax.legend()
ax.set_title("kinetic energy evolution as a function of time for varying Vth, dx=0.2")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta K$ (%)")
ax.set_yscale("log")

<ipython-input-576-e43593d5baa3>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [575]:
fig, ax = plt.subplots()
for path, K in K_vth.items():
    if "dx0.2" in path:
        continue
    it1, it2 = avg_interval(3,4, times_vth[path])
    K0= np.mean(K[it1:it2+1])    
    vth = path.split("/")[-1].strip("vth").split("dx")[0]
    ax.plot(times_vth[path], np.abs(K-K0)/K0*100, label="vth={}".format(vth))
ax.legend()
ax.set_title("kinetic energy evolution as a function of time for varying Vth, dx=0.1")
ax.set_xlabel("t")
ax.set_ylabel(r"$\Delta K$ (%)")
ax.set_yscale("log")

<ipython-input-575-e1b0aa0bdc03>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [588]:
vth0p2 = np.asarray([x[0] for x in vthdx if x[1] == 0.2])
vth0p1 = np.asarray([x[0] for x in vthdx if x[1] == 0.1])

K0 = {}
for path,K in K_vth.items():
        it1, it2 = avg_interval(3,4, times_vth[path])
        K0[path] = np.mean(K[it1:it2+1])
        

rel_K0p2 = np.asarray([np.abs(K[-1]-K0[path])/K0[path]*100 for path,K in K_vth.items() if "dx0.2" in path])
rel_K0p1 = np.asarray([np.abs(K[-1]-K0[path])/K0[path]*100 for path,K in K_vth.items() if "dx0.1" in path])
fig, ax = plt.subplots()
id2 = np.argsort(vth0p2)
id1 = np.argsort(vth0p1)
ax.plot(vth0p2[id2], rel_K0p2[id2], marker="o", label="dx = 0.2 (dt=0.002, 25k steps)")
ax.plot(vth0p1[id1], rel_K0p1[id1], marker="o", label="dx = 0.1 (dt=5e-4, 100k steps)")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(r"$\Delta K$ (%)")
ax.set_xlabel("Vth")
ax.set_title("kinetic energy evolution as a function of Vth")
ax.legend()

<ipython-input-588-a33e8b1997a2>:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [564]:
paths = glob("/home/aunai/phare_jobs/tests/conservation/run017/*vth*")
runs_vth = {}
Bnrj_vth = {}
K_vth_perp = {}
times_vth={}
vthdx = np.asarray(sorted([[float(x) for x in path.split("/")[-1].strip("vth").split("dx")] for path in paths],
               key=lambda x:x[1]))
paths = sorted(paths, key=lambda path: float(paths[0].split("/")[-1].strip("vth").split("dx")[1]))
for path in paths:
    print(path)
    runs_vth[path], Bnrj_vth[path], K_vth_perp[path], times_vth[path] = energies(path, kkind="perp")

/home/aunai/phare_jobs/tests/conservation/run017/vth0.01dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.05dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.1dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.3dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.5dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.75dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth1dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth2dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.05dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.01dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.1dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.3dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.5dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.75dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth1dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth1.5dx0.2
/home/aunai/phare_jobs/t

In [568]:
paths = glob("/home/aunai/phare_jobs/tests/conservation/run017/*vth*")
runs_vth = {}
Bnrj_vth = {}
K_vth_para = {}
times_vth={}
vthdx = np.asarray(sorted([[float(x) for x in path.split("/")[-1].strip("vth").split("dx")] for path in paths],
               key=lambda x:x[1]))
paths = sorted(paths, key=lambda path: float(paths[0].split("/")[-1].strip("vth").split("dx")[1]))
for path in paths:
    print(path)
    runs_vth[path], Bnrj_vth[path], K_vth_para[path], times_vth[path] = energies(path, kkind="para")

/home/aunai/phare_jobs/tests/conservation/run017/vth0.01dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.05dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.1dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.3dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.5dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.75dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth1dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth2dx0.1
/home/aunai/phare_jobs/tests/conservation/run017/vth0.05dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.01dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.1dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.3dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.5dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth0.75dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth1dx0.2
/home/aunai/phare_jobs/tests/conservation/run017/vth1.5dx0.2
/home/aunai/phare_jobs/t

In [582]:
vth0p2 = np.asarray([x[0] for x in vthdx if x[1] == 0.2])
vth0p1 = np.asarray([x[0] for x in vthdx if x[1] == 0.1])

K0 = {}
for path,K in K_vth_para.items():
        it1, it2 = avg_interval(3,4, times_vth[path])
        K0[path] = np.mean(K[it1:it2+1])
        
rel_K0p2_para= np.asarray([np.abs(K[-1]-K0[path])/K0[path]*100 for path, K in K_vth_para.items() if "dx0.2" in path])
rel_K0p1_para= np.asarray([np.abs(K[-1]-K0[path])/K0[path]*100 for path, K in K_vth_para.items() if "dx0.1" in path])
fig, ax = plt.subplots()
ax.plot(vth0p2, rel_K0p2_para, marker="o", label="PARA dx = 0.2 (dt=0.002, 25k steps)")
ax.plot(vth0p1, rel_K0p1_para, marker="o", label="PARA dx = 0.1 (dt=5e-4, 100k steps)")

ax.plot(vth0p2, rel_K0p2_perp, marker="o", label="PERP dx = 0.2 (dt=0.002, 25k steps)")
ax.plot(vth0p1, rel_K0p1_perp, marker="o", label="PERP dx = 0.1 (dt=5e-4, 100k steps)")


ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel(r"$\Delta K$ (%)")
ax.set_xlabel("Vth")
ax.set_title("para/perp kinetic energy evolution as a function of Vth")
ax.legend()
fig.tight_layout()

<ipython-input-582-29614f8775fc>:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …